# 08.4.线性回归的简洁实现
通过使用深度学习框架来简洁地实现线性回归模型生成数据集

**这段代码除了使用d2l生成了随机的训练集，其它部分都是可以在其它地方复用的**

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
print(features.shape)
print('-------------')
print(labels.shape)

torch.Size([1000, 2])
-------------
torch.Size([1000, 1])


调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train = True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)#*表示接受任意多个参数并将其放在一个元组中
    return data.DataLoader(dataset, batch_size, shuffle = is_train) #从dataset中随机取batch_size个元素并返回

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))#通过next函数得到一个X和一个y

[tensor([[-0.8043, -0.6604],
         [ 2.3337,  0.7962],
         [ 0.1063,  1.3691],
         [-0.2566, -0.5060],
         [-0.3702, -1.5404],
         [-1.8031, -0.4266],
         [ 0.7231, -0.8670],
         [-0.6818, -0.8323],
         [ 0.5636,  1.2887],
         [ 0.2712,  1.2440]]),
 tensor([[ 4.8527],
         [ 6.1428],
         [-0.2266],
         [ 5.4034],
         [ 8.7058],
         [ 2.0459],
         [ 8.5902],
         [ 5.6584],
         [ 0.9451],
         [ 0.5077]])]

使用框架的预定义好的层

In [3]:
# `nn`是神经网络的缩写
from torch import nn
"""
nn.Sequential创建了一个list of layers
输入是2维 输出是1维
"""
net = nn.Sequential(nn.Linear(2, 1))


初始化模型参数

In [4]:
#net[0]访问到nn.Linear(2, 1)
net[0].weight.data.normal_(0,0.01)#.weight访问到w，.data是真实data，normal_会使用正态分布替换掉data的值，两个参数是均值和方差
net[0].bias.data.fill_(0)#bais是偏差1
#上面两行相当于是在设置w和b w(向量)里各元素设置为均值为0，方差为0.01的随机数 b设置为0

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方范数

In [5]:
loss = nn.MSELoss()#均方损失已经被封装好了

实例化`SGD`实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) #小批量随机梯度下降也被包装好了
#net.parameters()获得所有参数包括`w`和`b` lr是学习率

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()# 更新参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')


epoch 1, loss 0.000286
epoch 2, loss 0.000097
epoch 3, loss 0.000098


In [8]:
# 查看最终的w和b
print(net[0].weight.data)
print(net[0].bias.data)

tensor([[ 1.9997, -3.4009]])
tensor([4.1988])
